# Import software libraries

In [1]:
import sys                                                      # Read system parameters.
import numpy as np                                              # Work with multi-dimensional arrays.
import pandas as pd
import sklearn                                                  # Train and evaluate machine learning models.
from sklearn.model_selection import train_test_split
import datasets                                                 # Format data.
from datasets import Dataset
import torch                                                    # Build deep learning models.
import transformers                                             # Work with transformer-based neural networks.
from transformers import AutoTokenizer, \
                         AutoModelForSeq2SeqLM, \
                         AutoModelForSequenceClassification, \
                         DataCollatorForSeq2Seq, \
                         pipeline, \
                         Seq2SeqTrainingArguments, \
                         Seq2SeqTrainer
import accelerate                                               # Abstract PyTorch code for multiple architectures.
import re                                                       # Work with regular expressions.
import warnings                                                 # Suppress warnings.
warnings.filterwarnings('ignore')

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- scikit-learn {}'.format(sklearn.__version__))
print('- Datasets {}'.format(datasets.__version__))
print('- PyTorch {}'.format(torch.__version__))
print('- Transformers {}'.format(transformers.__version__))
print('- Accelerate {}'.format(accelerate.__version__))

Libraries used in this project:
- Python 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]
- NumPy 1.24.3
- pandas 2.0.3
- scikit-learn 1.3.0
- Datasets 2.12.0
- PyTorch 2.2.0+cpu
- Transformers 4.32.1
- Accelerate 0.26.1


/home/student/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


# Read and preview the data

In [2]:
complaints_data = pd.read_pickle('data/consumer_loan_complaints_labeled.pickle')

complaints_data.head()

user_id  \
0  89a5f4ac-1042-458b-80c0-5f828ed0bd52   
1  7984b06e-0870-475c-b41f-a0b3205cdd78   
2  05315703-f1a7-4aae-88ba-1c47467aee2d   
3  cf7ee35b-dd93-4d66-a155-83473d53cf19   
4  cb8367da-0476-45cb-b911-143be6f86848   

                        Consumer complaint narrative  \
0  This Company uses many different names XXXX an...   
1  I did a pay off of {$9000.00} on Lending Club ...   
2  The account is a corporate account and now it ...   
3  Equifax credit reporting need to remove the Ba...   
4  Opened a line of credit with Fortiva Servicing...   

                                             Summary  
0  The company made unauthorized attempts to view...  
1  The company refused to provide documentation a...  
2  The company has placed a corporate account on ...  
3  The customer's bankruptcy event has not been r...  
4  The company closed the customer's credit line ...

# Prefix the complaint text

In [3]:
complaints_data['Consumer complaint narrative'] = 'summarize: ' + \
complaints_data['Consumer complaint narrative']

complaints_data.head()

user_id  \
0  89a5f4ac-1042-458b-80c0-5f828ed0bd52   
1  7984b06e-0870-475c-b41f-a0b3205cdd78   
2  05315703-f1a7-4aae-88ba-1c47467aee2d   
3  cf7ee35b-dd93-4d66-a155-83473d53cf19   
4  cb8367da-0476-45cb-b911-143be6f86848   

                        Consumer complaint narrative  \
0  summarize: This Company uses many different na...   
1  summarize: I did a pay off of {$9000.00} on Le...   
2  summarize: The account is a corporate account ...   
3  summarize: Equifax credit reporting need to re...   
4  summarize: Opened a line of credit with Fortiv...   

                                             Summary  
0  The company made unauthorized attempts to view...  
1  The company refused to provide documentation a...  
2  The company has placed a corporate account on ...  
3  The customer's bankruptcy event has not been r...  
4  The company closed the customer's credit line ...

# Format the dataset for compatibility

In [4]:
# Create lists from column data.
id_list = complaints_data['user_id'].tolist()
complaints_list = complaints_data['Consumer complaint narrative'].tolist()
summaries_list = complaints_data['Summary'].tolist()

# Create dictionary for compatibility.
data_dict = {'user_id': id_list,
             'Consumer complaint narrative': complaints_list,
             'Summary': summaries_list}

complaints_data = Dataset.from_dict(data_dict)

# Split the data into train and test sets

In [5]:
complaints_data = \
complaints_data.train_test_split(test_size = 0.2,
                                 seed = 24)

# Initialize a compatible tokenizer

In [6]:
pt_model = 't5-small'  # Pre-trained encoder–decoder model.
tokenizer = AutoTokenizer.from_pretrained(pt_model)

# Preprocess the dataset using the tokenizer

In [7]:
def preprocess(examples):
    model_inputs = tokenizer(examples['Consumer complaint narrative'],
                             max_length = 1024,
                             truncation = True)
    
    labels = tokenizer(text_target = examples['Summary'],
                       max_length = 128,
                       truncation = True)
    
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

In [8]:
complaints_data_tokenized = complaints_data.map(preprocess)

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

# Initialize a data collator

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer,
                                       model = pt_model)

# Initialize a seq2seq model

In [10]:
ft_model = AutoModelForSeq2SeqLM.from_pretrained(pt_model)

In [11]:
training_args = Seq2SeqTrainingArguments(output_dir = 'complaints_model')

trainer = Seq2SeqTrainer(model = ft_model,
                         args = training_args,
                         train_dataset = complaints_data_tokenized['train'],
                         eval_dataset = complaints_data_tokenized['test'],
                         tokenizer = tokenizer,
                         data_collator = data_collator)

# Train a fine-tuned summarization model

In [12]:
trainer.train()
trainer.save_model('complaints_model')

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


# Summarize a new complaint using the fine-tuned model

In [15]:
new_complaint = 'summarize: I called the local bank branch because GCNB ' \
                'closed my line of credit without warning. The customer-service ' \
                'person wasn\'t able to help me and redirected me to ' \
                'the account manager, who said there was suspicious activity ' \
                'on the account. I was forced to open a new account, which means ' \
                'I have to change my financial info on a bunch of different sites. ' \
                'This has been a terrible experience, and I\'m actively ' \
                'looking for another bank to do business with.'

summarizer = pipeline('summarization',
                      model = 'complaints_model',
                      max_length = 100)

summarizer(new_complaint)

[{'summary_text': "the customer-service person wasn't able to help me and redirected me to the account manager . GCNB closed my line of credit without warning ."}]